Загрузка данных без обработки для анализа данных 

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.models import model_from_json
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pylab import rcParams
rcParams['figure.figsize'] = 12, 10
pd.set_option('display.max_columns', 10)
np.set_printoptions(threshold=np.nan)


def categorization(data_frame_name, *columns):
    for column in columns:
        data_frame_name[column] = data_frame_name[column].astype('category').cat.codes.astype(int)


def neural_model_MLP():
    model = Sequential()
    model.add(Dense(49, input_shape=(7,), activation="relu", kernel_initializer="normal"))
    model.add(Dense(1, activation="sigmoid", kernel_initializer="normal"))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

data_df = pd.read_csv('StudentsPerformance.csv')

completed_df = data_df.loc[data_df['test preparation course'] == 'completed']
not_completed_df = data_df.loc[data_df['test preparation course'] == 'none']

completed_dict = dict()
completed_dict['completed'] = len(completed_df)
completed_dict['not_completed'] = len(not_completed_df)

plt.bar(range(len(completed_dict)), list(completed_dict.values()), align='center')
plt.xticks(range(len(completed_dict)), list(completed_dict.keys()))
plt.title("test preparation course total amount statistics ")
plt.ylabel("number of observations")
plt.show()

Наблюдаем, что выборка не репрезентативная, так как количество наблюдений разделенных по целевой переменной сильно отличается.
Для решения данной проблемы, далее отбрасываются лишние наблюдения для проведения более наглядного анализа. 

In [ ]:
completed_quantity = len(completed_df)
not_completed_df = not_completed_df.drop(not_completed_df.index[:-completed_quantity])

completed_dict = dict()
completed_dict['completed'] = len(completed_df)
completed_dict['not_completed'] = len(not_completed_df)

plt.bar(range(len(completed_dict)), list(completed_dict.values()), align='center')
plt.xticks(range(len(completed_dict)), list(completed_dict.keys()))
plt.title("test preparation course total amount statistics ")
plt.ylabel("number of observations")
plt.show()

Далее проанализируем некоторые переменные:

In [ ]:
gender_dict = dict()
gender_dict['female completed'] = len(completed_df.loc[completed_df['gender'] == 'female'])
gender_dict['female not_completed'] = len(not_completed_df.loc[not_completed_df['gender'] == 'female'])
gender_dict['male completed'] = len(completed_df.loc[completed_df['gender'] == 'male'])
gender_dict['male not_completed'] = len(not_completed_df.loc[not_completed_df['gender'] == 'male'])

plt.bar(range(len(gender_dict)), list(gender_dict.values()), align='center')
plt.xticks(range(len(gender_dict)), list(gender_dict.keys()))
plt.title("lunch total amount statistics ")
plt.ylabel("number of observations")
plt.show()

Анализ влияния пола студетна на целевую переменную: как видим, девушек в выборке несколько больше, однако влияния пола на целевую переменную не обнаружено 

In [ ]:
lunch_dict = dict()
lunch_dict['standard(completed)'] = len(completed_df.loc[completed_df['lunch'] == 'standard'])
lunch_dict['standard(not completed)'] = len(not_completed_df.loc[not_completed_df['lunch'] == 'standard'])
lunch_dict['free/reduced(completed)'] = len(completed_df.loc[completed_df['lunch'] == 'free/reduced'])
lunch_dict['free/reduced(not completed)'] = len(not_completed_df.loc[not_completed_df['lunch'] == 'free/reduced'])

plt.bar(range(len(lunch_dict)), list(lunch_dict.values()), align='center')
plt.xticks(range(len(lunch_dict)), list(lunch_dict.keys()))
plt.title("lunch total amount statistics ")
plt.ylabel("number of observations")
plt.show()

Наблюдаем небольшую корреляцию: Студенты с полным обедом чаще сдают тест, в то время, как с ограниченным обедом наоборот.

In [ ]:
education_dict_overal = dict()
education_dict_overal['bachelor\'s degree'] =\
    len(data_df.loc[data_df['parental level of education'] == 'bachelor\'s degree'])
education_dict_overal['master\'s degree'] =\
    len(data_df.loc[data_df['parental level of education'] == 'master\'s degree'])
education_dict_overal['associate\'s degree'] =\
    len(data_df.loc[data_df['parental level of education'] == 'associate\'s degree'])
education_dict_overal['some college'] =\
    len(data_df.loc[data_df['parental level of education'] == 'some college'])
education_dict_overal['high school'] =\
    len(data_df.loc[data_df['parental level of education'] == 'high school'])

plt.bar(range(len(education_dict_overal)), list(education_dict_overal.values()), align='center')
plt.xticks(range(len(education_dict_overal)), list(education_dict_overal.keys()))
plt.title("parental level of education total amount statistics ")
plt.ylabel("number of observations")
plt.show()

Распределение по ученым степеням.

Далее анализ сдачи теста по каждой ученой степени:

In [ ]:
education_dict_high = dict()
education_dict_high['high school(completed)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'high school'])
education_dict_high['high school(Not completed)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'high school'])

plt.bar(range(len(education_dict_high)), list(education_dict_high.values()), align='center')
plt.xticks(range(len(education_dict_high)), list(education_dict_high.keys()))
plt.title("parental level of education statistics")
plt.ylabel("number of observations")
plt.show()

education_dict_some = dict()
education_dict_some['some college(completed)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'some college'])
education_dict_some['some college(Not completed)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'some college'])

plt.bar(range(len(education_dict_some)), list(education_dict_some.values()), align='center')
plt.xticks(range(len(education_dict_some)), list(education_dict_some.keys()))
plt.title("parental level of education statistics")
plt.ylabel("number of observations")
plt.show()

education_dict_associate = dict()
education_dict_associate['associate\'s degree(completed)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'associate\'s degree'])
education_dict_associate['associate\'s degree (Not completed)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'associate\'s degree'])

plt.bar(range(len(education_dict_associate)), list(education_dict_associate.values()), align='center')
plt.xticks(range(len(education_dict_associate)), list(education_dict_associate.keys()))
plt.title("parental level of education statistics")
plt.ylabel("number of observations")
plt.show()

education_dict_master = dict()
education_dict_master['master\'s degree(completed)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'master\'s degree'])
education_dict_master['master\'s degree(Not completed)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'master\'s degree'])

plt.bar(range(len(education_dict_master)), list(education_dict_master.values()), align='center')
plt.xticks(range(len(education_dict_master)), list(education_dict_master.keys()))
plt.title("parental level of education statistics")
plt.ylabel("number of observations")
plt.show()

education_dict_bachelor = dict()
education_dict_bachelor['bachelor\'s degree(completed)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'bachelor\'s degree'])
education_dict_bachelor['bachelor\'s degree(Not completed)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'bachelor\'s degree'])

plt.bar(range(len(education_dict_bachelor)), list(education_dict_bachelor.values()), align='center')
plt.xticks(range(len(education_dict_bachelor)), list(education_dict_bachelor.keys()))
plt.title("parental level of education statistics")
plt.ylabel("number of observations")
plt.show()

Как видим: относительно общего количества студентов каждой ученой степени, меньше всего сдали студенты группы 'high school' и больше всего студенты группы 'bachelor's degree' и 'associate's degree', что докажем следующими графиками в относительных величинах:

In [ ]:
education_dict_comp = dict()
education_dict_comp['bachelor\'s degree(compl)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'bachelor\'s degree'])\
    / len(data_df.loc[data_df['parental level of education'] == 'bachelor\'s degree'])
education_dict_comp['master\'s degree(compl)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'master\'s degree'])\
    / len(data_df.loc[data_df['parental level of education'] == 'master\'s degree'])
education_dict_comp['associate\'s degree(compl)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'associate\'s degree'])\
    / len(data_df.loc[data_df['parental level of education'] == 'associate\'s degree'])
education_dict_comp['some college(compl)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'some college'])\
    / len(data_df.loc[data_df['parental level of education'] == 'some college'])
education_dict_comp['high school(compl)'] =\
    len(completed_df.loc[completed_df['parental level of education'] == 'high school'])\
    / len(data_df.loc[data_df['parental level of education'] == 'high school'])

plt.bar(range(len(education_dict_comp)), list(education_dict_comp.values()), align='center')
plt.xticks(range(len(education_dict_comp)), list(education_dict_comp.keys()))
plt.title("parental level of education statistics of completed")
plt.ylabel("dimensionless quantity")
plt.show()

education_dict_notcomp = dict()
education_dict_notcomp['bachelor\'s degree(not compl)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'bachelor\'s degree'])\
    / len(data_df.loc[data_df['parental level of education'] == 'bachelor\'s degree'])
education_dict_notcomp['master\'s degree(not compl)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'master\'s degree'])\
    / len(data_df.loc[data_df['parental level of education'] == 'master\'s degree'])
education_dict_notcomp['associate\'s degree(not compl)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'associate\'s degree'])\
    / len(data_df.loc[data_df['parental level of education'] == 'associate\'s degree'])
education_dict_notcomp['some college(not compl)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'some college'])\
    / len(data_df.loc[data_df['parental level of education'] == 'some college'])
education_dict_notcomp['high school(not compl)'] =\
    len(not_completed_df.loc[not_completed_df['parental level of education'] == 'high school'])\
    / len(data_df.loc[data_df['parental level of education'] == 'high school'])

plt.bar(range(len(education_dict_notcomp)), list(education_dict_notcomp.values()), align='center')
plt.xticks(range(len(education_dict_notcomp)), list(education_dict_notcomp.keys()))
plt.title("parental level of education statistics of Not completed")
plt.ylabel("dimensionless quantity")
plt.show()

Аналогичным образом можно проанализировать все остальные параметры, в том числе числовые, где можно наблюдать вид закона распределения значения переменной по всей выборке, дисперсию, медиану и моду {не успел:(}
однако учитывая полученную точность модели, смею предположить что явных зависимостей мы так и не увидим, данные достаточно однородны, и вклад в классификацию равномерно распределен по всем переменным (и он, крайне невелик)

Проведем простую подготовку данных: перевод категориальных переменных в числовые и дальнейшая нормализация всех данных, кроме целевой переменной.

In [ ]:
categorization(data_df, 'gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course')
data_df['target'] = data_df['test preparation course']
data_df.drop('test preparation course', axis=1, inplace=True)

seed = int(48)

df_values = data_df.values
y = df_values[:, 7]
x = df_values[:, 0:7].astype(float)
#x = preprocessing.scale(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
model_MLP = neural_model_MLP()
early_stopping = EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, mode='min')

accuracy_mlp = list()
for iteration in range(10):
    model_MLP.fit(x_train, y_train,
                  batch_size=10, epochs=10000, verbose=0, callbacks=[early_stopping])
    eval_MLP = model_MLP.evaluate(x_test, y_test, batch_size=400)
    accuracy_mlp.append(eval_MLP[1])
    print("MLP_%s: %.2f%%" % (model_MLP.metrics_names[1], eval_MLP[1]*100))

model_json = model_MLP.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model_MLP.save_weights("model.h5")

plt.plot(range(len(accuracy_mlp)), accuracy_mlp)
plt.show()

Как видим, модель дает невысокую точность и достаточно нестабильна. Так же испытывались более глубокие сети прямого распространения, однако они не дали весомого прироста, поэтому оставлена максимально простая архитектура для быстрого выполнения кода.


проведем окончательную, более точную валидацию модели (занимает 1-2 минуты)

In [ ]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=neural_model_MLP, epochs=50, batch_size=7, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, x, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))